In [ ]:
from dataclasses import dataclass
import time
import random
from pathlib import Path
import shutil

import tensorflow as tf
import numpy as np
from tqdm import tqdm
import yaml

from rl.network import ResNet
from rl.mcts import MCTS
from rl.buffer import ReplayBuffer
from rl import game

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

training_settings = config["training_settings"]
network_settings = config["network_settings"]
mcts_settings = config["mcts_settings"]
num_cpus = training_settings["num_cpus"]
n_episodes = training_settings["n_episodes"]
buffer_size = training_settings["buffer_size"]
batch_size = training_settings["batch_size"]
epochs_per_update = training_settings["epochs_per_update"]
update_period = training_settings["update_period"]
save_period = training_settings["save_period"]


@dataclass
class Sample:
    state: np.ndarray
    mcts_policy: np.ndarray
    reward: float


def selfplay(weights,current_episode, test=False):
    """Perform a self-play game and collect training data."""
    record = []
    if test:
        state = game.get_initial_test_state()
    else:
        state = game.get_initial_state()
    game.reset_used_columns()
    network = ResNet(action_space=game.ACTION_SPACE)

    # Initialize network parameters
    network.predict(game.encode_state(state))
    network.set_weights(weights)

    mcts = MCTS(network=network)
    mcts.current_episode = current_episode
    done = False
    total_score = 0
    step_count = 0
    prev_action = None

    while not done and step_count < game.MAX_STEPS:
        mcts_policy = mcts.search(
            root_state=state, num_simulations=mcts_settings["num_mcts_simulations"],prev_action=prev_action
        )
        if prev_action is not None:
            indices = [i for i in range(game.ACTION_SPACE) if i != prev_action]
            prob = mcts_policy[indices]
            prob = prob/ prob.sum()
            # if use_network_policy:
            #     prob = np.ones(len(prob))/len(prob)
            action = np.random.choice(indices, p=prob)
        else:
            indices = list(range(game.ACTION_SPACE))
            prob = mcts_policy
            # if use_network_policy:
            #     prob = np.ones(len(mcts_policy))/len(mcts_policy)
            action = np.random.choice(indices, p=prob)
        record.append(Sample(state.copy(), mcts_policy, reward=None))
        state, done = game.step(state, action, prev_action, mcts_policy)
        prev_action = action
        # print(state, action_score, done)
        # total_score += action_score
        step_count += 1


    # The reward is calculated based on the final state
    reward = game.get_reward(state, total_score)

    # Assign the reward to each sample
    for sample in record:
        sample.reward = reward
    return record

In [ ]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

training_settings = config["training_settings"]
network_settings = config["network_settings"]
mcts_settings = config["mcts_settings"]
num_cpus = training_settings["num_cpus"]
n_episodes = training_settings["n_episodes"]
buffer_size = training_settings["buffer_size"]
batch_size = training_settings["batch_size"]
epochs_per_update = training_settings["epochs_per_update"]
update_period = training_settings["update_period"]
save_period = training_settings["save_period"]

# ray.init(num_cpus=num_cpus, num_gpus=1, local_mode=False)

logdir = Path("log")
if logdir.exists():
    shutil.rmtree(logdir)
summary_writer = tf.summary.create_file_writer(str(logdir))

game.initialize_game()  # Initialize game variables

network = ResNet(action_space=game.ACTION_SPACE)

dummy_state = game.encode_state(game.get_initial_state())
network.predict(dummy_state)

current_weights = network.get_weights()

optimizer = tf.keras.optimizers.Adam(learning_rate=network_settings["learning_rate"])

replay = ReplayBuffer(buffer_size=buffer_size)

# # Start self-play workers
work_in_progresses = [selfplay(current_weights, True)]
n_updates = 0

n = 0
while n < n_episodes:
    for _ in tqdm(range(update_period)):
        # Wait for a self-play worker to finish
        finished = selfplay(current_weights,n, test=False)
        replay.add_record(finished)
        n += 1

    # Update network
    if len(replay) >= batch_size:
        num_iters = epochs_per_update * (len(replay) // batch_size)
        value_loss_weight = 1
        policy_loss_weight = 1
        for i in range(num_iters):
            states, mcts_policy, rewards = replay.get_minibatch(
                batch_size=batch_size
            )
            with tf.GradientTape() as tape:
                p_pred, v_pred = network(states, training=True)
                value_loss = tf.square(rewards - v_pred)
                policy_loss = -tf.reduce_sum(
                    mcts_policy * tf.math.log(p_pred + 1e-5), axis=1, keepdims=True
                )
                # print(f"{value_loss=}")
                # print(f"{policy_loss=}")
                # loss = tf.reduce_mean(value_loss + policy_loss)
                loss = tf.reduce_mean(value_loss_weight * value_loss + policy_loss_weight * policy_loss)
            grads = tape.gradient(loss, network.trainable_variables)
            grads, _ = tf.clip_by_global_norm(grads, 1.0)
            optimizer.apply_gradients(zip(grads, network.trainable_variables))
            n_updates += 1

            if i % 10 == 0:
                with summary_writer.as_default():
                    tf.summary.scalar(
                        "value_loss", tf.reduce_mean(value_loss), step=n_updates
                    )
                    tf.summary.scalar(
                        "policy_loss", tf.reduce_mean(policy_loss), step=n_updates
                    )

        current_weights = network.get_weights()

    if n % save_period == 0:
        network.save(f"checkpoints/network_{n}")
        network.save_weights(f"checkpoints/network_{n}.weights.h5")
        print("-" * 50)


In [ ]:
import numpy as np
import tensorflow as tf
from rl.network import ResNet
import rl.game as game

# ゲームの初期化
game.initialize_game()
game.reset_used_columns()

# ネットワークの構築と重みの読み込み
network = ResNet(action_space=game.ACTION_SPACE)
network = tf.keras.models.load_model("checkpoints/network_550")
# 初期状態の生成
# network.load_weights("checkpoints/network_210.weights.h5")

In [ ]:
for _ in range(10):
    state = game.get_initial_state()
    ans = []
    done = False
    total_score = 0
    step_count = 0
    prev_action = None
    print(state)
    while not done and step_count < game.MAX_STEPS:
        # 状態のエンコードと次元の調整
        encoded_state = game.encode_state(state)
        input_state = np.expand_dims(encoded_state, axis=0)
        
        # モデルによる予測
        policy_output, value_output = network.predict(input_state)
        policy = policy_output[0]
        if prev_action is not None:
            indices = [i for i in range(game.ACTION_SPACE) if i != prev_action]

            prob = policy[indices]
            action = np.random.choice(indices, p=prob/prob.sum())
        else:
            indices = list(range(game.ACTION_SPACE))
            action = np.random.choice(indices, p=policy)
        selected_action = game.ACTIONS[action]
        # print(f"Step {step_count}: Selected action {selected_action}")
        ans.append(selected_action)
        # アクションの適用
        state, done = game.step(state, action, prev_action)
        prev_action = action
        step_count += 1

        # 現在の状態を表示（必要に応じて）
        # print("Current state:\n", state)
        # print("Game done:", done)
        # print("-" * 50)

    # ゲーム結果の表示
    if done:
        print(f"Game finished successfully in {step_count} steps with {ans}")
    else:
        print(f"Game terminated after reaching the maximum steps ({game.MAX_STEPS}).")
        print(f"Total score: {total_score}")